# Demo: Callers

A `Caller` associates a `Prompt` with a specific LLM client and call parameters (assumes OpenAI-compatibility through a framework like `aisuite`).
This allows every Caller instance to use a different model and/or parameters, and sets expectations for the Caller instance.

Whereas `Prompts` validate _inputs_ to the template, `Callers` validate the LLM responses.

Since Callers leverage the LLM API directly, they can do things like function-calling / tool use.
If a tool-call instruction is detected, the Caller can try to `invoke` that call and return the function result as the response.

Additionally, Callers can be used as functions/tools in tool-calling workflows by leveraging Caller.signature() which denotes the inputs the Caller's Prompt requires.
Since a Caller has a specific client and model assigned, this effectively allows us to use Callers to route to specific models for specific use cases.
Since Callers can behave as functions themselves, we enable complex workflows where Callers can call Callers (ad infinitum ad nauseum).

- `ChatCaller` is a simple Caller implementation designed for chat messages without response validation.
- `RegexCaller` uses regex for response validation.
- `StructuredCaller` is intended for structured responses, and uses Pydantic for response validation.
- `ToolCaller` is a configuration for tool-use, and can optionally invoke the tool based on arguments in the LLM's response and return the function results.

In [ ]:
import json
import logging
import os
import re
import textwrap
from typing import cast

from pydantic import BaseModel, Field, create_model

import aisuite
import openai

from yaaal.core.caller import ChatCaller, RegexCaller, StructuredCaller, ToolCaller
from yaaal.core.prompt import (
    JinjaMessageTemplate,
    PassthroughMessageTemplate,
    Prompt,
    StaticMessageTemplate,
    StringMessageTemplate,
)
from yaaal.types.base import JSON
from yaaal.types.core import Conversation, Message
from yaaal.utilities import basic_log_config, format_json

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
basic_log_config()
logging.getLogger("yaaal").setLevel(logging.DEBUG)
logger = logging.getLogger(__name__).setLevel(logging.DEBUG)

## Quick Start

In [ ]:
# all Callers require a client and a model to call.
# `yaaal` is built around OpenAI-compatible APIs primarily provided by `aisuite`
client = aisuite.Client(
    provider_configs={
        "openai": {"api_key": os.environ["YAAAL_OPENAI_API_KEY"]},
        "anthropic": {"api_key": os.environ["YAAAL_ANTHROPIC_API_KEY"]},
        # ...
    }
)
# `aisuite` specifies models in "provider:model" format
model = "openai:gpt-4o-mini"

In [ ]:
# A `ChatCaller`
caller = ChatCaller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="chat",
        description="A simple chat",
        system_template=StaticMessageTemplate(role="system", template="You are a helpful assistant"),
        user_template=PassthroughMessageTemplate(),
    ),
)

2025-01-27 21:13:30,706 - DEBUG    - yaaal.core.caller - model:82 - All API requests for ChatCaller will use model : openai:gpt-4o-mini
2025-01-27 21:13:30,706 - DEBUG    - yaaal.core.caller - request_params:101 - All API requests for ChatCaller will use params : {'temperature': 0.7}


In [ ]:
# callers can still render conversations through their prompt
caller.prompt.render(user_vars={"content": "Who is Harry Potter?"})

{
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful assistant",
    },
    {
      "role": "user",
      "content": "Who is Harry Potter?",
    },
  ],
}

In [ ]:
# callers are called as functions to get the response from the LLM
response = caller(system_vars=None, user_vars={"content": "Who is Harry Potter?"})

print(textwrap.fill(response, replace_whitespace=False))

2025-01-27 21:13:35,657 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:35,659 - DEBUG    - yaaal.core.caller - _handle_response:172 - Response object has message.content
2025-01-27 21:13:35,659 - DEBUG    - yaaal.core.caller - _validate_content:216 - Using default (passthrough) validator.


Harry Potter is a fictional character and the protagonist of the
"Harry Potter" series, which consists of seven fantasy novels written
by British author J.K. Rowling. The story follows Harry, a young boy
who discovers he is a wizard on his eleventh birthday. He attends
Hogwarts School of Witchcraft and Wizardry, where he learns magic,
makes friends, and faces various challenges.

The series explores
themes of friendship, bravery, love, and the battle between good and
evil, particularly through Harry's ongoing conflict with the dark
wizard Lord Voldemort, who killed Harry's parents and seeks to conquer
the wizarding world. The books include "Harry Potter and the
Philosopher's Stone" (published as "Harry Potter and the Sorcerer's
Stone" in the U.S.), "Harry Potter and the Chamber of Secrets," "Harry
Potter and the Prisoner of Azkaban," "Harry Potter and the Goblet of
Fire," "Harry Potter and the Order of the Phoenix," "Harry Potter and
the Half-Blood Prince," and "Harry Potter and the De

In [ ]:
# callers have a `signature` method that uses the prompt signature
print(format_json(caller.signature().model_json_schema()))

{
  "$defs": {
    "PassthroughModel": {
      "properties": {
        "content": {
          "title": "Content",
          "type": "string",
        },
      },
      "required": [
        "content",
      ],
      "title": "PassthroughModel",
      "type": "object",
    },
  },
  "description": "A simple chat",
  "properties": {
    "user_vars": {
      "$ref": "#/$defs/PassthroughModel",
    },
  },
  "required": [
    "user_vars",
  ],
  "title": "chat",
  "type": "object",
}


In [ ]:
# A `RegexCaller` validates the response with a regex pattern
pattern = re.compile(r"\b[A-E]\b(?!.*\b[A-E]\b)")

template_str = """
"The following are multiple choice questions (with answers) about Star Wars.

What is the model designation of an X-Wing?
A. T-65B
B. BTL-A4
C. RZ-1
D. A/SF-01
Answer: A

{{question}}
Answer:
""".strip()


class MCQAQuestion(BaseModel):
    question: str = Field(description="The multiple choice question")


regex_caller = RegexCaller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="chat",
        description="Multiple-choice question answering",
        system_template=JinjaMessageTemplate(role="system", template=template_str, template_vars_model=MCQAQuestion),
    ),
    response_validator=pattern,
)

2025-01-27 21:13:35,689 - DEBUG    - yaaal.core.caller - model:82 - All API requests for RegexCaller will use model : openai:gpt-4o-mini
2025-01-27 21:13:35,690 - DEBUG    - yaaal.core.caller - request_params:101 - All API requests for RegexCaller will use params : {'temperature': 0.7}


In [ ]:
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = regex_caller(system_vars={"question": question}, user_vars=None)

if response == "E":
    print("Success! 🎉")
# print(textwrap.fill(response, replace_whitespace=False))

2025-01-27 21:13:36,211 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:36,212 - DEBUG    - yaaal.core.caller - _handle_response:172 - Response object has message.content
2025-01-27 21:13:36,212 - DEBUG    - yaaal.core.caller - _validate_content:292 - Validating response against regex pattern.


Success! 🎉


In [ ]:
# A `StructuredCaller` validates the response with a Pydantic model, and is good for structure data extraction
class Person(BaseModel, extra="ignore"):
    name: str
    age: int
    favorite_color: str


# Use an fstring to create a jinja prompt --
# The fstring allows us to substitute in the Person schema.
# Because we're using fstrings, we have to double the `{}`
# so python understands they do not indicate an fstring substitution.
template_str = f"""
Identify facts about a person as they introduce themselves.

Respond in a format that matches the following schema:

<schema>
{Person.model_json_schema()}
</schema>

<introduction>
{{{{introduction}}}}
</introduction>
""".strip()


class PersonIntroduction(BaseModel):
    introduction: str


structured_caller = StructuredCaller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="person details",
        description="Identify details about a person",
        system_template=JinjaMessageTemplate(
            role="system",
            template=template_str,
            template_vars_model=PersonIntroduction,
        ),
    ),
    response_validator=Person,
)

2025-01-27 21:13:36,234 - DEBUG    - yaaal.core.caller - model:82 - All API requests for StructuredCaller will use model : openai:gpt-4o-mini
2025-01-27 21:13:36,235 - DEBUG    - yaaal.core.caller - request_params:101 - All API requests for StructuredCaller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'Person', 'strict': True, 'parameters': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'favorite_color': {'title': 'Favorite Color', 'type': 'string'}}, 'required': ['name', 'age', 'favorite_color'], 'title': 'Person', 'type': 'object', 'additionalProperties': False}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'Person'}}}


In [ ]:
introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

response = structured_caller(system_vars={"introduction": introduction}, user_vars=None)

print(type(response))
print(format_json(response.model_dump()))

2025-01-27 21:13:36,663 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:36,665 - DEBUG    - yaaal.core.caller - _handle_response:176 - Response object has message.tool_call(s), using first.
2025-01-27 21:13:36,665 - DEBUG    - yaaal.core.caller - _validate_tool:456 - Validating tool_call response against response_validator Pydantic model (Person).


<class '__main__.Person'>
{
  "name": "Bob",
  "age": 42,
  "favorite_color": "blue",
}


In [ ]:
# A `ToolCaller` can choose to call tools or respond like a normal LLM.

template_str = """Use the best tool for the task.""".strip()

tool_caller = ToolCaller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    prompt=Prompt(
        name="tool use",
        description="Determine which tool to use",
        system_template=StaticMessageTemplate(role="system", template=template_str),
        user_template=PassthroughMessageTemplate(),
    ),
    toolbox=[regex_caller, structured_caller],  # we can use other callers as tools!
    auto_invoke=True,  # we should actually make the recommended tool call
)

2025-01-27 21:13:36,680 - DEBUG    - yaaal.core.caller - model:82 - All API requests for ToolCaller will use model : openai:gpt-4o-mini
2025-01-27 21:13:36,683 - DEBUG    - yaaal.core.caller - request_params:101 - All API requests for ToolCaller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'chat', 'strict': True, 'parameters': {'$defs': {'MCQAQuestion': {'properties': {'question': {'description': 'The multiple choice question', 'title': 'Question', 'type': 'string'}}, 'required': ['question'], 'title': 'MCQAQuestion', 'type': 'object', 'additionalProperties': False}}, 'description': 'Multiple-choice question answering', 'properties': {'system_vars': {'$ref': '#/$defs/MCQAQuestion'}}, 'required': ['system_vars'], 'title': 'chat', 'type': 'object', 'additionalProperties': False}, 'description': 'Multiple-choice question answering'}}, {'type': 'function', 'function': {'name': 'person_details', 'strict': True, 'parameters': {'$defs': {'PersonIn

In [ ]:
# the tool_caller will automatically add the tools to the request parameters
print(format_json(tool_caller.request_params))

{
  "temperature": 0.7,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "chat",
        "strict": true,
        "parameters": {
          "$defs": {
            "MCQAQuestion": {
              "properties": {
                "question": {
                  "description": "The multiple choice question",
                  "title": "Question",
                  "type": "string",
                },
              },
              "required": [
                "question",
              ],
              "title": "MCQAQuestion",
              "type": "object",
              "additionalProperties": false,
            },
          },
          "description": "Multiple-choice question answering",
          "properties": {
            "system_vars": {
              "$ref": "#/$defs/MCQAQuestion",
            },
          },
          "required": [
            "system_vars",
          ],
          "title": "chat",
          "type": "object",
          "additionalPro

In [ ]:
# this should call the person schema tool

introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

response = tool_caller(
    system_vars=None,
    user_vars={"content": introduction},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

2025-01-27 21:13:37,905 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:37,906 - DEBUG    - yaaal.core.caller - _handle_response:176 - Response object has message.tool_call(s), using first.
2025-01-27 21:13:37,907 - DEBUG    - yaaal.core.caller - _validate_tool:627 - Validating tool call against tool signature.
2025-01-27 21:13:38,413 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:38,413 - DEBUG    - yaaal.core.caller - _handle_response:176 - Response object has message.tool_call(s), using first.
2025-01-27 21:13:38,414 - DEBUG    - yaaal.core.caller - _validate_tool:456 - Validating tool_call response against response_validator Pydantic model (Person).


<class 'yaaal.types.core.ToolMessage'>
{
  "role": "tool",
  "content": "{"name":"Bob","age":42,"favorite_color":"blue"}",
  "tool_call_id": "call_ZRBT5cJFQAzc3kcOoKFoAsfj",
}


In [ ]:
# this should call the Star Wars QA tool
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = tool_caller(
    system_vars=None,
    user_vars={"content": question},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

2025-01-27 21:13:40,257 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:40,258 - DEBUG    - yaaal.core.caller - _handle_response:176 - Response object has message.tool_call(s), using first.
2025-01-27 21:13:40,258 - DEBUG    - yaaal.core.caller - _validate_tool:627 - Validating tool call against tool signature.
2025-01-27 21:13:40,668 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:40,669 - DEBUG    - yaaal.core.caller - _handle_response:172 - Response object has message.content
2025-01-27 21:13:40,670 - DEBUG    - yaaal.core.caller - _validate_content:292 - Validating response against regex pattern.


<class 'yaaal.types.core.ToolMessage'>
{
  "role": "tool",
  "content": "E",
  "tool_call_id": "call_VLvac7G4NLr35XKVoOm2kSFi",
}


In [ ]:
# this should just respond without calling a tool
response = tool_caller(
    system_vars=None,
    user_vars={"content": "Hello world!"},
)

print(type(response))
print(format_json(response.model_dump() if isinstance(response, BaseModel) else response))

2025-01-27 21:13:41,328 - DEBUG    - yaaal.core.caller - _chat_completions_create:163 - Converting response object to ChatCompletion
2025-01-27 21:13:41,330 - DEBUG    - yaaal.core.caller - _handle_response:172 - Response object has message.content
2025-01-27 21:13:41,330 - DEBUG    - yaaal.core.caller - _validate_content:216 - Using default (passthrough) validator.


<class 'str'>
"Hello! How can I assist you today?"
